In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
from tensorflow.keras.models import Sequential

In [6]:
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization

In [7]:
from tensorflow.keras.layers import Conv2D,MaxPooling2D

In [8]:
import os

In [9]:
num_classes = 5
img_rows,img_cols = 48,48
batch_size = 32


In [10]:
train_data_dir = r'/content/drive/My Drive/project/train5'
validation_data_dir = r'/content/drive/My Drive/project/validation5'


In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,rotation_range=30,shear_range=0.3,zoom_range=0.3,width_shift_range=0.4,height_shift_range=0.4,horizontal_flip=True,fill_mode='nearest')

In [12]:
validation_datagen = ImageDataGenerator(rescale=1./255)


In [13]:
train_generator = train_datagen.flow_from_directory(train_data_dir,color_mode='grayscale',target_size=(img_rows,img_cols),batch_size=batch_size,class_mode='categorical',shuffle=True)

Found 24351 images belonging to 5 classes.


In [14]:
validation_generator = validation_datagen.flow_from_directory(validation_data_dir,color_mode='grayscale',
                                                              target_size=(img_rows,img_cols),
                                                              batch_size=batch_size,
                                                              class_mode='categorical',
                                                              shuffle=True)



Found 5937 images belonging to 5 classes.


In [15]:
model = Sequential()

In [16]:
# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


In [17]:
# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [18]:
# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


In [19]:
# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))


In [20]:
# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))


In [21]:
# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [22]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 48, 48, 32)        320       
_________________________________________________________________
activation (Activation)      (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 24, 24, 32)        0

In [23]:
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau


In [24]:
checkpoint = ModelCheckpoint('Emotion_detection.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)


In [25]:
earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

In [26]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2,
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)


In [27]:
callbacks = [earlystop,checkpoint,reduce_lr]

In [28]:
model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])


In [29]:
nb_train_samples = 24351
nb_validation_samples = 5937
epochs=10


In [31]:
history=model.fit_generator(
                train_generator,
                steps_per_epoch=nb_train_samples//batch_size,
                epochs=epochs,
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=nb_validation_samples//batch_size)


Epoch 1/10
207/207 [==============================] - ETA: 0s - loss: 1.5446 - accuracy: 0.2954
Epoch 00001: val_loss improved from 1.51024 to 1.49321, saving model to Emotion_detection.h5
207/207 [==============================] - 202s 975ms/step - loss: 1.5446 - accuracy: 0.2954 - val_loss: 1.4932 - val_accuracy: 0.3354
Epoch 2/10
207/207 [==============================] - ETA: 0s - loss: 1.5453 - accuracy: 0.2984
Epoch 00002: val_loss improved from 1.49321 to 1.48360, saving model to Emotion_detection.h5
207/207 [==============================] - 196s 946ms/step - loss: 1.5453 - accuracy: 0.2984 - val_loss: 1.4836 - val_accuracy: 0.3354
Epoch 3/10
207/207 [==============================] - ETA: 0s - loss: 1.5340 - accuracy: 0.3071
Epoch 00003: val_loss improved from 1.48360 to 1.47640, saving model to Emotion_detection.h5
207/207 [==============================] - 195s 940ms/step - loss: 1.5340 - accuracy: 0.3071 - val_loss: 1.4764 - val_accuracy: 0.3552
Epoch 4/10
207/207 [========